this notebook was used to create a standartized inventory for pico, including the spiketimes and psth h5 files. additionally, the information at hand was extracted from the excel file and saved.

In [3]:
import numpy as np
import io
import pandas as pd
import os
import fnmatch
import shutil
import re
from ruamel.yaml import YAML


In [4]:
def find_directories_with_name(root_dir, directory_name):
    directory_paths = []
    for foldername, subfolders, _ in os.walk(root_dir):
        if directory_name in subfolders:
            directory_paths.append(os.path.join(foldername, directory_name))
    return directory_paths

############### Find Directories of SpikeTimes ################################
###############################################################################

parent_directory = '/braintree/data2/active/users/sgouldin/projects'
target_directory_name = 'spikeTime'
spikeTime_directories = find_directories_with_name(parent_directory, target_directory_name)

spike_parent = []
for path in spikeTime_directories:
    spike_parent.append(os.path.join(*path.split('/')[0:10]))

In [5]:
def get_date(date):
    """
    Get the Dates from the Excel file.
    """

    try:
        dates = date.split('/')
        if len(dates[-1])>2:year = dates[-1]
        else: year=f'20{dates[-1]}'
        if len(dates[0])>1: month = dates[0]
        else: month = f'0{dates[0]}'
        if len(dates[1])>1:day=dates[1]
        else:day=f'0{dates[1]}'
        

    except:
        try:
            dates = str(date).split()[0].split('-')
            if len(dates)==3:
                year=dates[0]
                month=dates[1]
                if len(dates[2])>1:day=dates[2]
                else:day=f'0{dates[2]}'
            else:
                pass
        except:
            pass

    try: return day, month, year
    except: return 0,0,0
    

def find_files_with_pattern(root_dir, pattern):
    """
    Find Associated .h5 File for each SpikeTime Folder.
    """
    return [os.path.join(foldername, filename) for foldername, subfolders, filenames in os.walk(root_dir) for filename in filenames if filename.startswith(pattern) and filename.endswith('.h5')]


############### Create Storage Dir and Load Excel File ########################
###############################################################################
   
storage_dir = '/braintree/home/aliya277/inventory'
try: os.mkdir(storage_dir)
except: pass
os.chdir('/braintree/home/aliya277')
path = 'dandi_brainscore/Pipeline Monkey Schedule New.xlsx'
data = pd.read_excel(path, sheet_name='pico')
new_header = data.iloc[0]  
data = data[1:]       
data.columns = new_header  
data = data.fillna('empty')  


for spike in (spikeTime_directories):

    ############### Get Recording Info from FileName ##############################
    ###############################################################################
    ind = spike.split('/').index('intanproc')
    SubjectName     = spike.split('/')[ind-1]
    ImageSetName_v1 = spike.split('/')[ind-3]
    Date            = re.findall('..',spike.split('/')[ind+1].split('_')[-2])
    year            = f'20{Date[0]}'
    month           = Date[1]
    day             = Date[2]
    SessionTime     = spike.split('/')[ind+1].split('_')[-1]

    
    if SubjectName == 'pico':


        ############### Filter by Date in FileName and ExcelDate ##########################
        ###############################################################################
        for date,index in zip(data['Date'], range(1,len(data))):
            exl_day, exl_month, exl_year = get_date(date)

            if exl_day+exl_month+exl_year == day+month+year:


                ############### Filter by FileName and ExcelStimulus ##########################
                ###############################################################################
                list = [data['Stimuli'][index]]
                for i in range(1,10):
                    try:
                        if data['Date'][index+i] == 'empty':
                            list.append(data['Stimuli'][index+i])
                            i+=1
                        else:
                            break
                    except:break
                found_item = False

                for item, i in zip(list, range(len(list))): 
                    if item.lower() in ImageSetName_v1:
                        ImageSetName = data['Stimuli'][index+i]
                        # StimOnnOff   = data['ON/OFF (ms)'][index+i]
                        RecInfo      = data.iloc[[index+i-1]].to_dict('records')[0]
                        found_item = True

                if found_item == False:
                    for item, i in zip(list, range(len(list))):
                        spike_new = ImageSetName_v1.lower().replace('-', '_').split('_')
                        item_new = item.lower().replace('-', '_').split('_') 

                        if set(item_new).intersection(set(spike_new)) == set(item_new) or set(spike_new).intersection(set(item_new)) == set(spike_new):
                            ImageSetName = data['Stimuli'][index+i]
                            # StimOnnOff   = data['ON/OFF (ms)'][index+i]
                            RecInfo      = data.iloc[[index+i-1]].to_dict('records')[0]
                            found_item = True
                
                ############### Create Standartized Directory Name ############################
                ###############################################################################
                if found_item == True:
                    SessionDate = f'{year}{month}{day}'
                    if ImageSetName == 'Normalizers':
                        directory = f'norm_FOSS.sub_{SubjectName}.{SessionDate}_{SessionTime}.proc'
                    elif ImageSetName == 'Normalizers-HVM':
                        directory = f'norm_HVM.sub_{SubjectName}.{SessionDate}_{SessionTime}.proc'
                    else: 
                        directory = f'exp_{ImageSetName}.sub_{SubjectName}.{SessionDate}_{SessionTime}.proc'

                    try: os.mkdir(os.path.join(storage_dir, directory))
                    except: pass


                    ############### Copy Files in Respective Inventory Directory ##################
                    ###############################################################################
                    matching_h5files = find_files_with_pattern(os.path.join('/', *spike.split('/')[0:10]), f'{Date[0]}{Date[1]}{Date[2]}')
                    
                    # shutil.copytree(spike, os.path.join(storage_dir, directory, 'SpikeTimes'))
                    # if matching_h5files:
                    #     for files in matching_h5files:
                    #         try: os.mkdir(os.path.join(storage_dir, directory, 'h5Files'))
                    #         except: pass
                    #         shutil.copy2(files, os.path.join(storage_dir, directory, 'h5Files', files.split('/')[-1]))

                    ############### Create .txt File with Relevant Info for NWB ###################
                    ###############################################################################
                    
                    RecInfo['intanproc'] = spike
                    RecInfo['SessionDate'] = SessionDate
                    RecInfo['SessionTime'] = SessionTime
                    yaml = YAML()
                    with open(os.path.join(storage_dir, directory,"RecInfo.yaml"), 'w') as yamlfile:
                        yaml.dump((RecInfo), yamlfile)


                    
                else:
                    print('Wrong Names: ', month, day, year, list, '\t', spike.split('/')[ind-3], index)
                    pass
                    


In [ ]:
def find_files_with_extension(root_dir, extension):
    file_paths = []
    for foldername, subfolders, filenames in os.walk(root_dir):
        for filename in fnmatch.filter(filenames, f'*.{extension}'):
            file_paths.append(os.path.join(foldername, filename))
    return file_paths


############### Find Directories for PSTH h5 Files ############################
###############################################################################
parent_directory = '/braintree/data2/active/users/sgouldin/projects'
extension = 'h5'
h5_files = find_files_with_extension(parent_directory, extension)

h5_parent = []
for path in h5_files:
    h5_parent.append(os.path.join(*path.split('/')[0:10]))

############### Compare SpikeTimes Directoriy and h5 Directory ################
###############################################################################
spikeTime_set = set(spike_parent)
h5_set = set(h5_parent)

# Find the directories that are unique to each set
unique_to_spikeTime = spikeTime_set - h5_set
unique_to_h5 = h5_set - spikeTime_set

# Check if the sets are different
if len(unique_to_spikeTime) > 0 or len(unique_to_h5) > 0:
    print("The sets are different.")
else:
    print("The sets are the same.")

# Print the unique directories
print("Directories unique to 'spikeTime':")
print(unique_to_spikeTime)

print("Directories unique to 'h5':")
print(unique_to_h5)


############### Manually Correct if Needed  ###################################
###############################################################################
# manually copying one file:
# for dir in unique_to_other:
#     if dir.split('/')[-1] == 'pico':
#         print(dir)
#         file = os.listdir(os.path.join('/'+dir, 'h5'))[0]
#         shutil.copy2(os.path.join('/'+dir, 'h5', file ), '/braintree/home/aliya277/inventory/exp_robustness_guy_d0_v7_chong_dryrun.sub_pico.20230717_130030.proc/h5Files')